# DB2-Salesforce connector: Tools mapping to users

In [154]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Name'
object_id = 'ContactToolAssociation__c'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"mathisfun992728tXYwCb5yMm6XtohbTRw8ZdBbi",    
} 
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

In [3]:
import sys
sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')
    
import pandas as pd
import time
import datetime

from DB2SalesforceAPI import DB2SalesforceAPI

## Obtain tool information from DB2 

In [199]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub_metrics' \
                                               %(sql_login_params['username'], sql_login_params['password']))

# toolstart
sql_query = "select user, tool as instance from nanohub_metrics.toolstart"

toolstart_df = pd.read_sql_query(sql_query, engine)

# tool_version
sql_query = "select toolname, instance from nanohub.jos_tool_version"

tool_version_df = pd.read_sql_query(sql_query, engine)

In [200]:
# convert all to lower case
toolstart_df['user'] = toolstart_df['user'].apply(str.lower)
toolstart_df['instance'] = toolstart_df['instance'].apply(str.lower)

tool_version_df['toolname'] = tool_version_df['toolname'].apply(str.lower)
tool_version_df['instance'] = tool_version_df['instance'].apply(str.lower)

# display

display(toolstart_df.head(2))
display(tool_version_df.head(2))

,user,instance
0,yxc5571,qclab_r79
1,yxc5571,qclab_r79


,toolname,instance
0,112016a,112016a_dev
1,1dbtetransient,1dbtetransient_dev


In [201]:
# insert the toolname into instance as well
dummy = pd.DataFrame()
dummy['toolname'] = tool_version_df['toolname'].drop_duplicates()
dummy['instance'] = dummy['toolname']

tool_version_df = pd.concat([dummy, tool_version_df], axis=0).drop_duplicates().reset_index(drop=True)

# display
tool_version_df.head(3)

,toolname,instance
0,112016a,112016a
1,1dbtetransient,1dbtetransient
2,1dchainmd,1dchainmd


In [202]:
# merge toolname into toolstart
df = pd.merge(toolstart_df, tool_version_df, how='left', left_on='instance', right_on='instance')[['user', 'toolname']]

In [222]:
# count number of tool runs by each user, each tool
df = df.groupby(['user', 'toolname'])['toolname'].count().to_frame(name='cnt').reset_index()

In [223]:
# just in case some NaN values made through
df = df[~df.user.isna()]
df = df[~df.toolname.isna()]

# display
df.head(3)

,user,toolname,cnt
0,123whdxor,bjt,2
1,99033653ak,cntbands-ext,2
2,aa469635295,bandstrlab,67


## Obtain Salesforce IDs

In [161]:
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [162]:
# query the Salesforce IDs for contacts and citations. when updating junction objects, these IDs must be used

# get Salesforce ID for contacts
sf_username_df = db_s.query_data('SELECT Id, nanoHUB_username__c FROM Contact where nanoHUB_username__c != NULL')

# display
sf_username_df.head(3)

[Success] Bulk job creation successful. Job ID = 7505w00000LN8y4AAD
{"id":"7505w00000LN8y4AAD","operation":"query","object":"Contact","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-05-04T01:09:23.000+0000","systemModstamp":"2020-05-04T01:09:23.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000LN8y4AAD","operation":"query","object":"Contact","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-05-04T01:09:23.000+0000","systemModstamp":"2020-05-04T01:09:31.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":216165,"retries":0,"totalProcessingTime":7192}
[Success] Bulk job completed successfully.


,Id,nanoHUB_username__c
0,0035w000031Vsp1AAC,hubrepo
1,0035w000031Vsp2AAC,support
2,0035w000031Vsp3AAC,gridstat


In [163]:
# get Salesforce ID for citations
sf_tool_df = db_s.query_data('SELECT Id, Tool_name__c FROM nanoHUB_tools__c')

# display
sf_tool_df.head(3)

[Success] Bulk job creation successful. Job ID = 7505w00000LN8nNAAT
{"id":"7505w00000LN8nNAAT","operation":"query","object":"nanoHUB_tools__c","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-05-04T01:09:37.000+0000","systemModstamp":"2020-05-04T01:09:37.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000LN8nNAAT","operation":"query","object":"nanoHUB_tools__c","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-05-04T01:09:37.000+0000","systemModstamp":"2020-05-04T01:09:38.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":1588,"retries":0,"totalProcessingTime":283}
[Success] Bulk job completed successfully.


,Id,Tool_name__c
0,a0s5w00000k5MOwAAM,ellipsom
1,a0s5w00000k5MOxAAM,hydrolab
2,a0s5w00000k5MOyAAM,huckel


## Match data with Salesforce format

In [224]:
# make sure all strings are lower cases

sf_username_df['nanoHUB_username__c'] = sf_username_df['nanoHUB_username__c'].astype('str').apply(str.lower)

sf_tool_df['Tool_name__c'] = sf_tool_df['Tool_name__c'].astype('str').apply(str.lower)

In [225]:
# merge SF citation and contact IDs into user-citation DF
ct_tolink_df = pd.merge(df, sf_tool_df, how='inner', left_on='toolname', right_on='Tool_name__c')\
                           .rename(columns={'Id':'SF_ID_tool'})
                                            
ct_tolink_df = pd.merge(ct_tolink_df, sf_username_df, how='inner', left_on='user', right_on='nanoHUB_username__c')\
                           .rename(columns={'Id':'SF_ID_contact'})

# display
ct_tolink_df.head(2)

,user,toolname,cnt,SF_ID_tool,Tool_name__c,SF_ID_contact,nanoHUB_username__c
0,123whdxor,bjt,2,a0s5w00000k5MRwAAM,bjt,0035w000034IqqqAAC,123whdxor
1,alskdjfh6161,bjt,9,a0s5w00000k5MRwAAM,bjt,0035w000034IqPuAAK,alskdjfh6161


In [226]:
# create a new column for object name
ct_tolink_df['Name'] = ct_tolink_df.apply(lambda x: '%s_%s'%(x.nanoHUB_username__c, x.Tool_name__c), axis=1)

In [227]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf['Name']         = ct_tolink_df['Name']
df_sf['Contact__c'] = ct_tolink_df['SF_ID_contact']
df_sf['Tool__c']       = ct_tolink_df['SF_ID_tool']
df_sf['Total_Runs__c'] = ct_tolink_df['cnt']
sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
Name,123whdxor_bjt,alskdjfh6161_bjt
Contact__c,0035w000034IqqqAAC,0035w000034IqPuAAK
Tool__c,a0s5w00000k5MRwAAM,a0s5w00000k5MRwAAM
Total_Runs__c,2,9


## To Salesforce Sales Cloud CRM

In [228]:
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# specify Salesforce object ID and external ID
db_s.object_id = object_id
db_s.external_id = external_id

Obtained Salesforce access token ...... True


In [229]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000LNAjBAAX
[Success] CSV upload successful. Job ID = 7505w00000LNAjBAAX
[Success] Closing job successful. Job ID = 7505w00000LNAjBAAX


In [231]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_status())

{'apexProcessingTime': 0,
 'apiActiveProcessingTime': 1136,
 'apiVersion': 47.0,
 'columnDelimiter': 'COMMA',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'createdById': '0055w00000ArpYvAAJ',
 'createdDate': '2020-05-04T05:24:12.000+0000',
 'externalIdFieldName': 'Name',
 'id': '7505w00000LNAjBAAX',
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'numberRecordsFailed': 0,
 'numberRecordsProcessed': 352,
 'object': 'ContactToolAssociation__c',
 'operation': 'upsert',
 'retries': 0,
 'state': 'JobComplete',
 'systemModstamp': '2020-05-04T05:24:17.000+0000',
 'totalProcessingTime': 1248}
